In [97]:
!nvidia-smi

Fri Dec 24 11:56:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:86:00.0 Off |                    0 |
| N/A   33C    P0    55W / 300W |  11265MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Multi-task training

Inspired by this notebook: https://colab.research.google.com/github/zphang/zphang.github.io/blob/master/files/notebooks/Multi_task_Training_with_Transformers_NLP.ipynb

In [7]:
# import tensorflow as tf
# import torch
# print(tf.__version__)
# print(torch.__version__)
# !pip install pandas==1.1.4
# !pip install tensorflow==2.4.1
# !pip install torch==1.8.1

In [62]:
# import libraries and define utility functions
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, ClassLabel, load_metric
import transformers
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer



def create_datasets(tokenized_lyrics, feature, n=None, verbose=True, n_class=1, max_val=1):
    """
    n_class : if provided, quantize to this many classes with equal amount of per class
    n : to subset for testing purposes
    """
    lyrics_lab = tokenized_lyrics.copy()
    
    if n_class > 1:
        edges = histedges_equalN(train[feature].values, n_class, max_val=max_val)
        if verbose:
            print(edges)
        lyrics_lab["train"] = lyrics_lab["train"].add_column("labels", quantize_feature(train[feature], edges))
        lyrics_lab["validation"] = lyrics_lab["validation"].add_column("labels", quantize_feature(val[feature], edges))
        lyrics_lab["test"] = lyrics_lab["test"].add_column("labels", quantize_feature(test[feature], edges))
    else:
        lyrics_lab["train"] = lyrics_lab["train"].add_column("labels", train[feature])
        lyrics_lab["validation"] = lyrics_lab["validation"].add_column("labels", val[feature])
        lyrics_lab["test"] = lyrics_lab["test"].add_column("labels", test[feature])
    if verbose:
        print(lyrics_lab)
    
    if n is not None:
        lyrics_lab["train"] = lyrics_lab["train"].shuffle(seed=seed).select(range(n)) 
        lyrics_lab["validation"] = lyrics_lab["validation"].shuffle(seed=seed).select(range(n)) 
        lyrics_lab["test"] = lyrics_lab["test"].shuffle(seed=seed).select(range(n)) 
    
    if verbose:
        print(lyrics_lab["train"])
        print(lyrics_lab["validation"])
        print(lyrics_lab["test"])
            
    return lyrics_lab


def quantize_feature(values, edges):
    labels = values.copy().astype(np.int)
    for i, _edge in enumerate(edges[:-1]):
        if i == 0:
            mask = values < edges[i+1]
        else:
            mask = np.logical_and(values >= edges[i], values < edges[i+1])
        labels[mask] = i
    return labels


def histedges_equalN(x, nbin, max_val=None):
    # https://stackoverflow.com/a/39419049
    npt = len(x)
    edges = np.interp(np.linspace(0, npt, nbin + 1), np.arange(npt), np.sort(x))
    if max_val:
        edges[-1] = max_val
    return edges

Our base model for fine-tuning.

In [8]:
model_str = "sentence-transformers/all-mpnet-base-v2"    # 418 MB, 384 max length
# model_str = "sentence-transformers/all-distilroberta-v1"   # 292 MB, 512 max length
# model_str = "sentence-transformers/all-MiniLM-L6-v2"       # 80 MB, 256 max length
# model_str = "sentence-transformers/all-MiniLM-L12-v2"       # 118 MB, 256 max length

# 1) load and create datasets

In [9]:
# load original dataframe
df_clean = pd.read_pickle('df_clean_v4_14122021_py35.pkl')
print(df_clean.shape)
df_clean.head()

(15863, 17)


,lyrics,explicit,song_name,song_popularity,mode,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,release_year,artist,genre
song_id,,,,,,,,,,,,,,,,,
3e9HZxeyfWwjeyPAMmWSSQ,Thought I'd end up with Sean. But he wasn't a ...,1,"thank u, next",86,1,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,2019,Ariana Grande,dance pop
5p7ujcrUXASCNwRaWNHR1C,Found you when your heart was broke. I filled ...,1,Without Me,87,1,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,2018,Halsey,dance pop
2xLMifQCjDGFmkHkpNLD9h,"Astro, yeah. Sun is down, freezin' cold. That'...",1,SICKO MODE,85,1,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,2018,Travis Scott,hip-hop/rap
1rqqCSm0Qe4I9rUvWncaom,"High, high hopes. Had to have high, high hopes...",0,High Hopes,86,1,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,2018,Panic! At The Disco,rock
0bYg9bo50gSsH3LtXe2SQn,I-I-I don't want a lot for Christmas. There is...,0,All I Want for Christmas Is You,63,1,0.16400,0.335,0.625,0.000000,0.0708,-7.462,0.0386,0.346,150.277,1994,Mariah Carey,dance pop


In [10]:
# train, validation, test split
seed = 11
val_size = 0.15
test_size = 0.15
train, _eval = train_test_split(df_clean, test_size=val_size + test_size, random_state=seed)
val, test = train_test_split(df_clean, test_size=test_size / (val_size + test_size), random_state=seed)
print(train.shape)
print(val.shape)
print(test.shape)

(11104, 17)
(7931, 17)
(7932, 17)


In [11]:
# create a DatasetDict as expected by Hugging Face
lyrics = DatasetDict()

# arguments expected by forward: https://github.com/huggingface/transformers/blob/9aeacb58bab321bc21c24bbdf7a24efdccb1d426/src/transformers/modeling_bert.py#L1313
lyrics["train"] = Dataset.from_dict({"text": list(train["lyrics"])})
lyrics["validation"] = Dataset.from_dict({"text": list(val["lyrics"])})
lyrics["test"] = Dataset.from_dict({"text": list(test["lyrics"])})

print(lyrics)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 11104
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 7931
    })
    test: Dataset({
        features: ['text'],
        num_rows: 7932
    })
})


In [12]:
# tokenize
tokenizer = AutoTokenizer.from_pretrained(model_str, use_fast=True)
    
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_lyrics = lyrics.map(preprocess_function, batched=True)

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [31]:
# create genre dataset
labels_lst = sorted(list(set(df_clean["genre"])))
labels = ClassLabel(names=labels_lst)
genre_dataset = tokenized_lyrics.copy()
genre_dataset["train"] = genre_dataset["train"].add_column("labels", labels.str2int(train["genre"]))
genre_dataset["validation"] = genre_dataset["validation"].add_column("labels", labels.str2int(val["genre"]))
genre_dataset["test"] = genre_dataset["test"].add_column("labels", labels.str2int(test["genre"]))
# cast to ClassLabel
for _key in genre_dataset.keys():
    new_features = genre_dataset[_key].features.copy()
    new_features["labels"] = labels
    genre_dataset[_key] = genre_dataset[_key].cast(new_features)
# check correct type
genre_dataset["train"].features

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

{'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'text': Value(dtype='string', id=None),
 'labels': ClassLabel(num_classes=8, names=['acoustic/folk', 'country', 'dance pop', 'hip-hop/rap', 'pop', 'r&b', 'rock', 'soul/disco'], names_file=None, id=None)}

In [53]:
# create dataset for other metadata
valence_dataset = create_datasets(tokenized_lyrics, feature="valence", n_class=2, verbose=False)
danceability_dataset = create_datasets(tokenized_lyrics, feature="danceability", n_class=2, verbose=False)
energy_dataset = create_datasets(tokenized_lyrics, feature="energy", n_class=2, verbose=False)


_id = 1

print(f"{train.iloc[_id]['song_name']} by {train.iloc[_id]['artist']}\n")
print(f"Lyrics : {valence_dataset['train'][_id]['text'][:300]}")
print(f"Genre : {labels.int2str(genre_dataset['train'][_id]['labels'])}")
print(f"Valence : {valence_dataset['train'][_id]['labels']}")
print(f"Danceability : {danceability_dataset['train'][_id]['labels']}")
print(f"Energy : {energy_dataset['train'][_id]['labels']}")


Wrap Your Arms Around Me by KC & The Sunshine Band

Lyrics : Wrap your arms around me. Come on, come on. Wrap your arms around me. Get it on, get it on. Wrap your arms around me. Come on, come on. Wrap your arms around me. Rock me all night long. Oh, don't waste a minute. Don't waste no time. Give right to it. Come on and be mine. Wrap your arms around me. Co
Genre : soul/disco
Valence : 1
Danceability : 1
Energy : 1


In [54]:
dataset_dict = {
    "genre": genre_dataset,
    "valence": valence_dataset,
#     "danceability": danceability_dataset,
#     "energy": energy_dataset,
}

In [96]:
columns_dict = {
    "genre": ['input_ids', 'attention_mask', 'labels'],
    "valence": ['input_ids', 'attention_mask', 'labels'],
    "danceability": ['input_ids', 'attention_mask', 'labels'],
    "energy": ['input_ids', 'attention_mask', 'labels'],
}


features_dict = {}
for task_name, dataset in dataset_dict.items():
    features_dict[task_name] = {}
    for phase, phase_dataset in dataset.items():
#         features_dict[task_name][phase] = phase_dataset.map(
#             convert_func_dict[task_name],
#             batched=True,
#             load_from_cache_file=False,
#         )
        features_dict[task_name][phase] = phase_dataset
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))
        features_dict[task_name][phase].set_format(
            type="torch", 
            columns=columns_dict[task_name],
        )
        print(task_name, phase, len(phase_dataset), len(features_dict[task_name][phase]))

genre train 11104 11104
genre train 11104 11104
genre validation 7931 7931
genre validation 7931 7931
genre test 7932 7932
genre test 7932 7932
valence train 11104 11104
valence train 11104 11104
valence validation 7931 7931
valence validation 7931 7931
valence test 7932 7932
valence test 7932 7932


In [92]:
features_dict["genre"]["train"]["labels"].dtype

torch.int64

# 2) create multitask model

In [63]:
class MultitaskModel(transformers.PreTrainedModel):
    def __init__(self, encoder, taskmodels_dict):
        """
        Setting MultitaskModel up as a PretrainedModel allows us
        to take better advantage of Trainer features
        """
        super().__init__(transformers.PretrainedConfig())

        self.encoder = encoder
        self.taskmodels_dict = nn.ModuleDict(taskmodels_dict)

    @classmethod
    def create(cls, model_name, model_type_dict, model_config_dict):
        """
        This creates a MultitaskModel using the model class and config objects
        from single-task models. 

        We do this by creating each single-task model, and having them share
        the same encoder transformer.
        """
        shared_encoder = None
        taskmodels_dict = {}
        for task_name, model_type in model_type_dict.items():
            model = model_type.from_pretrained(
                model_name, 
                config=model_config_dict[task_name],
            )
            if shared_encoder is None:
                shared_encoder = getattr(model, cls.get_encoder_attr_name(model))
            else:
                setattr(model, cls.get_encoder_attr_name(model), shared_encoder)
            taskmodels_dict[task_name] = model
        return cls(encoder=shared_encoder, taskmodels_dict=taskmodels_dict)

    @classmethod
    def get_encoder_attr_name(cls, model):
        """
        The encoder transformer is named differently in each model "architecture".
        This method lets us get the name of the encoder attribute
        """
        model_class_name = model.__class__.__name__
        print(model_class_name)
        if model_class_name.startswith("Bert"):
            return "bert"
        elif model_class_name.startswith("Roberta"):
            return "roberta"
        elif model_class_name.startswith("Albert"):
            return "albert"
        elif model_class_name.startswith("MPNet"):
            return "mpnet"
        else:
            raise KeyError(f"Add support for new model {model_class_name}")

    def forward(self, task_name, **kwargs):
        return self.taskmodels_dict[task_name](**kwargs)
    
multitask_model = MultitaskModel.create(
    model_name=model_str,
    model_type_dict={
        "genre": transformers.AutoModelForSequenceClassification,
        "valence": transformers.AutoModelForSequenceClassification
    },
    model_config_dict={
        "genre": transformers.AutoConfig.from_pretrained(model_str, num_labels=len(labels_lst)),
        "valence": transformers.AutoConfig.from_pretrained(model_str, num_labels=2)
    },
)

Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a

MPNetForSequenceClassification


Some weights of the model checkpoint at sentence-transformers/all-mpnet-base-v2 were not used when initializing MPNetForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-mpnet-base-v2 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a

MPNetForSequenceClassification


In [68]:
# check use same encoder
if "mpnet" in model_str:
    print(multitask_model.encoder.embeddings.word_embeddings.weight.data_ptr())
    for _key in multitask_model.taskmodels_dict.keys():
        print(multitask_model.taskmodels_dict[_key].mpnet.embeddings.word_embeddings.weight.data_ptr())

1694497280
1694497280
1694497280


# 3) preparing multi-task data loader
https://github.com/huggingface/transformers/blob/master/src/transformers/data/data_collator.py

In [77]:
DataCollator    # indeed a function

<function typing.NewType.<locals>.new_type(x)>

In [78]:
import dataclasses
from torch.utils.data.dataloader import DataLoader
from transformers.data.data_collator import DataCollator, InputDataClass, DefaultDataCollator
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler
from typing import List, Union, Dict


# class NLPDataCollator(DataCollator):
class NLPDataCollator(DefaultDataCollator):
    """
    Extending the existing DataCollator to work with NLP dataset batches
    """
    def collate_batch(self, features):
        first = features[0]
        if isinstance(first, dict):
            # NLP data sets current works presents features as lists of dictionary
            # (one per example), so we  will adapt the collate_batch logic for that
            if "labels" in first and first["labels"] is not None:
                if first["labels"].dtype == torch.int64:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.long)
                else:
                    labels = torch.tensor([f["labels"] for f in features], dtype=torch.float)
                batch = {"labels": labels}
            for k, v in first.items():
                if k != "labels" and v is not None and not isinstance(v, str):
                    batch[k] = torch.stack([f[k] for f in features])
            return batch
        else:
            # otherwise, revert to using the default collate_batch
            return DefaultDataCollator().collate_batch(features)


class StrIgnoreDevice(str):
    """
    This is a hack. The Trainer is going call .to(device) on every input
    value, but we need to pass in an additional `task_name` string.
    This prevents it from throwing an error
    """
    def to(self, device):
        return self


class DataLoaderWithTaskname:
    """
    Wrapper around a DataLoader to also yield a task name
    """
    def __init__(self, task_name, data_loader):
        self.task_name = task_name
        self.data_loader = data_loader

        self.batch_size = data_loader.batch_size
        self.dataset = data_loader.dataset

    def __len__(self):
        return len(self.data_loader)
    
    def __iter__(self):
        for batch in self.data_loader:
            batch["task_name"] = StrIgnoreDevice(self.task_name)
            yield batch


class MultitaskDataloader:
    """
    Data loader that combines and samples from multiple single-task
    data loaders.
    """
    def __init__(self, dataloader_dict):
        self.dataloader_dict = dataloader_dict
        self.num_batches_dict = {
            task_name: len(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        self.task_name_list = list(self.dataloader_dict)
        self.dataset = [None] * sum(
            len(dataloader.dataset) 
            for dataloader in self.dataloader_dict.values()
        )

    def __len__(self):
        return sum(self.num_batches_dict.values())

    def __iter__(self):
        """
        For each batch, sample a task, and yield a batch from the respective
        task Dataloader.

        We use size-proportional sampling, but you could easily modify this
        to sample from some-other distribution.
        """
        task_choice_list = []
        for i, task_name in enumerate(self.task_name_list):
            task_choice_list += [i] * self.num_batches_dict[task_name]
        task_choice_list = np.array(task_choice_list)
        np.random.shuffle(task_choice_list)
        dataloader_iter_dict = {
            task_name: iter(dataloader) 
            for task_name, dataloader in self.dataloader_dict.items()
        }
        for task_choice in task_choice_list:
            task_name = self.task_name_list[task_choice]
            yield next(dataloader_iter_dict[task_name])  

# 4) preparing multi-task Trainer

In [80]:
class MultitaskTrainer(transformers.Trainer):

    def get_single_train_dataloader(self, task_name, train_dataset):
        """
        Create a single-task data loader that also yields task names
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")
#         if is_tpu_available():
#             train_sampler = get_tpu_sampler(train_dataset)
#         else:
        train_sampler = (
            RandomSampler(train_dataset)
            if self.args.local_rank == -1
            else DistributedSampler(train_dataset)
        )

        data_loader = DataLoaderWithTaskname(
            task_name=task_name,
            data_loader=DataLoader(
              train_dataset,
              batch_size=self.args.train_batch_size,
              sampler=train_sampler,
              collate_fn=self.data_collator.collate_batch,
            ),
        )

#         if is_tpu_available():
#             data_loader = pl.ParallelLoader(
#                 data_loader, [self.args.device]
#             ).per_device_loader(self.args.device)
        return data_loader

    def get_train_dataloader(self):
        """
        Returns a MultitaskDataloader, which is not actually a Dataloader
        but an iterable that returns a generator that samples from each 
        task Dataloader
        """
        return MultitaskDataloader({
            task_name: self.get_single_train_dataloader(task_name, task_dataset)
            for task_name, task_dataset in self.train_dataset.items()
        })

# 5) Train!

In [93]:
train_dataset = {
    task_name: dataset["train"] 
    for task_name, dataset in dataset_dict.items()
}
trainer = MultitaskTrainer(
    model=multitask_model,
    args=transformers.TrainingArguments(
        output_dir="genre_and_valence_fine_tuning",
        overwrite_output_dir=True,
        learning_rate=1e-5,
        do_train=True,
        num_train_epochs=3,
        # Adjust batch size if this doesn't fit on the Colab GPU
        per_device_train_batch_size=8,  
        save_steps=3000,
    ),
    data_collator=NLPDataCollator(),
    train_dataset=train_dataset,
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 22208
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 8328


Step,Training Loss


KeyboardInterrupt: 